In [12]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.display import display
from IPython.display import clear_output
from ipyfilechooser import FileChooser
from scipy import stats

%matplotlib qt
wt = FileChooser('C:/Users/LAB-ADMIN/Desktop/pipeline-program/analysis_results')
wt.filter_pattern = ['wt_*.csv']
wt.title = "WT distribution"
wt_dist = None

output = widgets.Output()


def wt_selection(chooser):
    with output:
        display(f"Selected: {chooser.selected_filename}")


wt.register_callback(wt_selection)

title_selector = widgets.Text(value='', placeholder='Title to use in plot', description='')

cmp = FileChooser('C:/Users/LAB-ADMIN/Desktop/pipeline-program/analysis_results')
cmp.filter_pattern = ['wt_*.csv']
cmp.title = "Distribution to compare with"
cmp_dist = None

voxel_size = 7.536e-6

def cmp_selection(chooser):
    if len(wt.selected) == 0:
        with output:
            display("Select WT first, since selecting the comparison runs the comparison algorithms.")
            display(wt.selected)
    else:
        with output:
            clear_output()
            display(f"Statistics for {chooser.selected_filename}...")
            conditions = ["wt", "ncr1", "npc2"]
            dist_list = []
            for condition in conditions:
                df1 = pd.read_csv(f"{wt.selected_path}/{condition}{wt.selected_filename[2:]}", index_col=False).iloc[:,
                      1:]
                df2 = pd.read_csv(f"{chooser.selected_path}/{condition}{chooser.selected_filename[2:]}",
                                  index_col=False).iloc[:, 1:]
                dist_list.append(np.array(df1)[:, 0] * voxel_size)
                dist_list.append(np.array(df2)[:, 0] * voxel_size)

            # get x values to plot
            min_x = min(np.amin(dist) for dist in dist_list)
            max_x = max(np.amax(dist) for dist in dist_list)
            dif = max_x - min_x
            xx = np.linspace(min_x - dif / 10, max_x + dif / 10, 10000)

            # create plot
            fig, axes = plt.subplots(figsize=(8 * len(conditions), 6), nrows=1, ncols=3)

            cols = ["blue","orange","green"]
            for i, cond in enumerate(conditions):
                kde1 = stats.gaussian_kde(dist_list[i * 2])
                kde2 = stats.gaussian_kde(dist_list[(i * 2) + 1])
                y1 = kde1(xx)
                y2 = kde2(xx)
                axes[i].plot(xx, y1, color=cols[i], label=f"{cond.upper()} unconsumed")
                axes[i].plot([], [], ' ', label=f"mean volume {np.mean(dist_list[i * 2]).round(decimals=3)} µm³")
                axes[i].plot([], [], ' ', label=" ")
                axes[i].plot(xx, y2, "--", color="olive", label=f"{cond.upper()} consumed")
                axes[i].plot([], [], ' ', label=f"mean volume {np.mean(dist_list[(i * 2) + 1]).round(decimals=3)} µm³")
                #axes[i].fill_between(xx,y1,y2)

                axes[i].set_xlabel('volume µm³')
                axes[i].set_ylabel('density')
                #axes[i].set_title(f"{cond.upper()}")
                axes[i].legend()
            
            #fig.suptitle("Consumed vs unconsumed volume distribution in S. cerevisiae")
            plt.show()



cmp.register_callback(cmp_selection)

display(wt)
display(cmp)
display(title_selector)
display(output)

FileChooser(path='C:\Users\LAB-ADMIN\Desktop\pipeline-program\analysis_results', filename='', title='WT distri…

FileChooser(path='C:\Users\LAB-ADMIN\Desktop\pipeline-program\analysis_results', filename='', title='Distribut…

Text(value='', placeholder='Title to use in plot')

Output()